In [9]:
from fairseq_utils import preprocess_series, load_dataset, load_model, get_embeddings
from fairseq.data import Dictionary
from constants import TOKENIZER_SUFFIXES, TOKENIZER_PATH, FAIRSEQ_PREPROCESS_PATH, PROJECT_PATH, PREDICTION_MODEL_PATH, TASK_PATH
import pandas as pd

molecules = ["CCC", "CC"]

def embed_all(path, cuda=0):
    output_dict = dict()
    for model_type in ["bart","roberta"]:
        tokenizer_dict = dict()
        for tokenizer_suffix in TOKENIZER_SUFFIXES:
            tokenizer_dict[tokenizer_suffix] = embed(path, model_type, tokenizer_suffix, cuda)
        output_dict[model_type] = tokenizer_dict 
    return output_dict

def embed(path, model_type, tokenizer_suffix, cuda):
    model_suffix = tokenizer_suffix+"_"+model_type
    fairseq_dict_path = TASK_PATH / "bbbp" /tokenizer_suffix
    model_path = PREDICTION_MODEL_PATH/model_suffix/"checkpoint_last.pt"
    model = load_model(model_path,fairseq_dict_path,str(cuda))
    dataset_path = (path / tokenizer_suffix/ "input0")
    dataset = load_dataset(dataset_path/"train")
    source_dictionary = Dictionary.load(str(dataset_path/"dict.txt"))
    embeddings = get_embeddings(model, dataset, source_dictionary, whole_mol=True, cuda=cuda)
    return embeddings
    
latent_geometry_path = PROJECT_PATH/"latent_space_geometry"
preprocess_series(molecules, latent_geometry_path)
embedding_dict = embed_all(latent_geometry_path, 1)

100%|████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 132.78it/s]
